In [79]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tflearn
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
%matplotlib inline
print(tf.__version__)
tf.logging.set_verbosity("DEBUG")
sess = tf.InteractiveSession()
file_writer = tf.summary.FileWriter('models/logs', sess.graph)

1.4.0


In [92]:
# define preprocessing function
def preprocess(disease, first, last, data, label):
    for count in range(first,last+1):
        string = "processed/" + str(disease) + "/" + str(count) + ".jpg"
        pic = Image.open(string).load()
        rgb = []
        
        red = []
        green=[]
        blue=[]
        for x in range(0,64):
            for y in range(0,64):
                #chroma = [0, 0, 0]
                #_sum = pic[x,y][0] + pic[x,y][1] + pic[x,y][2]
                #chroma[0] = pic[x,y][0]/_sum
                #chroma[1] = pic[x,y][1]/_sum
                #chroma[2] = pic[x,y][2]/_sum
                #rgb.append(chroma)
                red.append(pic[x,y][0])
                green.append(pic[x,y][1])
                blue.append(pic[x,y][2])

        #data.append(rgb)
        rgb.append(red)
        rgb.append(green)
        rgb.append(blue)
        
        data.append(rgb)
        
        if disease == "acne":
            label.append(0)
        elif disease == "ezcema":
            label.append(1)
        elif disease == "psoriasis":
            label.append(2)
        elif disease == "seborrheic keratoses":
            label.append(0)
        elif disease == "skin cancer":
            label.append(1)

# define training data
train_data = []
train_labels = []

preprocess("acne",1,48,train_data,train_labels)
preprocess("ezcema",1,51,train_data,train_labels)
#preprocess("psoriasis",1,44,train_data,train_labels)
#preprocess("seborrheic keratoses",1,45,train_data,train_labels)
#preprocess("skin cancer",1,45,train_data,train_labels)
    
train_data = np.asarray(train_data, dtype=np.float32)
train_labels = np.asarray(train_labels)

# define testing data
test_data = []
test_labels = []

preprocess("acne",49,53,test_data,test_labels)
preprocess("ezcema",52,56,test_data,test_labels)
#preprocess("psoriasis",45,49,test_data,test_labels)
#preprocess("seborrheic keratoses",46,50,test_data,test_labels)
#preprocess("skin cancer",46,50,test_data,test_labels)

test_data = np.asarray(test_data, dtype=np.float32)
test_labels = np.asarray(test_labels)

In [93]:
test_data.shape

(10, 3, 4096)

In [80]:
# define network architecture
def cnn_model_fn1(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[13, 13],
      padding="same",
      activation=tf.nn.relu) #output: [-1, 64, 64, 64]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2) #output [-1, 32, 32, 64]
    
    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=96,
      kernel_size=[13, 13],
      padding="same",
      activation=tf.nn.relu) #output: [-1, 32, 32, 96]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2) #output [-1, 16, 16, 96]
    
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 96])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Loss Calculation
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # Configure training op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add Evaluation Metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# define network architecture
def cnn_model_fn2(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[16, 16],
      padding="same",
      activation=tf.nn.relu) #output: [-1, 64, 64, 64]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2) #output [-1, 32, 32, 64]
    #print(conv1.shape, pool1.shape)
    
    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=128,
      kernel_size=[12, 12],
      padding="same",
      activation=tf.nn.relu) #output: [-1, 32, 32, 128]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2) #output [-1, 16, 16, 128]
    #print(conv2.shape, pool2.shape)
    
    # Dense Layers
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 128])
    dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout1 = tf.layers.dropout(inputs=dense1, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=dropout1, units=512, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(inputs=dense2, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout2, units=4)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Loss Calculation
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=4)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # Configure training op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add Evaluation Metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [89]:
# Create dermacam model
# there are 2 models, one is the original one, the other is the new one im experimenting with

dermacam = tf.estimator.Estimator(model_fn=cnn_model_fn1,model_dir="models/dermacam1")
#dermacam = tf.estimator.Estimator(model_fn=cnn_model_fn2,model_dir="models/dermacam2")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
#tensors_to_log = {"classes": "classess"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=20)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/dermacam1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f46f5bbe320>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [94]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=50,
    num_epochs=None,
    shuffle=True)

dermacam.train(
    input_fn=train_input_fn,
    steps=61,
    hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into models/dermacam1/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1 into models/dermacam1/model.ckpt.


INFO:tensorflow:probabilities = [[ 1.          0.00000001]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.00000001]
 [ 1.          0.        ]
 [ 0.99932873  0.00067135]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 0.10558987  0.89441013]
 [ 1.          0.        ]
 [ 1.          0.00000005]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.00000001]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 0.04759699  0.95240307]
 [ 0.99999988  0.00000009]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.      

...

INFO:tensorflow:loss = 11.0531, step = 1


INFO:tensorflow:loss = 11.0531, step = 1


INFO:tensorflow:probabilities = [[ 0.48681659  0.51318347]
 [ 0.48769274  0.51230729]
 [ 0.48875612  0.51124388]
 [ 0.48727098  0.51272899]
 [ 0.48826373  0.51173627]
 [ 0.48919183  0.51080817]
 [ 0.48660243  0.51339763]
 [ 0.48721531  0.51278472]
 [ 0.48741448  0.51258546]
 [ 0.48483166  0.51516831]
 [ 0.48753437  0.5124656 ]
 [ 0.48477185  0.51522815]
 [ 0.48829946  0.51170051]
 [ 0.48658383  0.51341623]
 [ 0.48970643  0.5102936 ]
 [ 0.48755842  0.51244158]
 [ 0.48742378  0.51257628]
 [ 0.48786283  0.51213723]
 [ 0.48834595  0.51165414]
 [ 0.48573804  0.51426196]
 [ 0.48759291  0.51240706]
 [ 0.48792318  0.51207691]
 [ 0.48800179  0.51199818]
 [ 0.48668268  0.51331741]
 [ 0.48726091  0.51273906]
 [ 0.48803484  0.51196516]
 [ 0.48730221  0.51269782]
 [ 0.48728713  0.51271278]
 [ 0.4893485   0.51065147]
 [ 0.48576924  0.51423073]
 [ 0.48837498  0.51162511]
 [ 0.48549962  0.51450032]
 [ 0.48690569  0.51309425]
 [ 0.48550949  0.51449049]
 [ 0.48896885  0.51103115]
 [ 0.48773095  0.512269

INFO:tensorflow:probabilities = [[ 0.48681659  0.51318347]
 [ 0.48769274  0.51230729]
 [ 0.48875612  0.51124388]
 [ 0.48727098  0.51272899]
 [ 0.48826373  0.51173627]
 [ 0.48919183  0.51080817]
 [ 0.48660243  0.51339763]
 [ 0.48721531  0.51278472]
 [ 0.48741448  0.51258546]
 [ 0.48483166  0.51516831]
 [ 0.48753437  0.5124656 ]
 [ 0.48477185  0.51522815]
 [ 0.48829946  0.51170051]
 [ 0.48658383  0.51341623]
 [ 0.48970643  0.5102936 ]
 [ 0.48755842  0.51244158]
 [ 0.48742378  0.51257628]
 [ 0.48786283  0.51213723]
 [ 0.48834595  0.51165414]
 [ 0.48573804  0.51426196]
 [ 0.48759291  0.51240706]
 [ 0.48792318  0.51207691]
 [ 0.48800179  0.51199818]
 [ 0.48668268  0.51331741]
 [ 0.48726091  0.51273906]
 [ 0.48803484  0.51196516]
 [ 0.48730221  0.51269782]
 [ 0.48728713  0.51271278]
 [ 0.4893485   0.51065147]
 [ 0.48576924  0.51423073]
 [ 0.48837498  0.51162511]
 [ 0.48549962  0.51450032]
 [ 0.48690569  0.51309425]
 [ 0.48550949  0.51449049]
 [ 0.48896885  0.51103115]
 [ 0.48773095  0.512269

KeyboardInterrupt: 

In [84]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=True)

eval_results = dermacam.evaluate(input_fn=eval_input_fn,hooks=[logging_hook])
print(eval_results)

# 1 1 1 1 1 1 1 1 1 1(Predicted Results)
# 0 0 0 0 0 1 1 1 1 1 (Actual results)

INFO:tensorflow:Starting evaluation at 2017-11-19-10:52:03
INFO:tensorflow:Restoring parameters from models/dermacam1/model.ckpt-61
INFO:tensorflow:probabilities = [[ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]
 [ 0.46699971  0.53300029]]
INFO:tensorflow:Finished evaluation at 2017-11-19-10:52:07
INFO:tensorflow:Saving dict for global step 61: accuracy = 0.5, global_step = 61, loss = 0.69533
{'accuracy': 0.5, 'loss': 0.69532996, 'global_step': 61}


In [78]:
dd = tf.estimator.Exporter()
dd.export(estimator=dermacam,export_path="models/logs",checkpoint_path="models/dermacam1",eval_result=eval_results,is_the_final_export=False)